In [18]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

In [19]:
model_name = "csebuetnlp/mT5_m2m_crossSum_enhanced"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [20]:
from torchinfo import summary

summary(model)

Layer (type:depth-idx)                                       Param #
MT5ForConditionalGeneration                                  --
├─Embedding: 1-1                                             192,086,016
├─MT5Stack: 1-2                                              192,086,016
│    └─Embedding: 2-1                                        (recursive)
│    └─ModuleList: 2-2                                       --
│    │    └─MT5Block: 3-1                                    7,079,808
│    │    └─MT5Block: 3-2                                    7,079,424
│    │    └─MT5Block: 3-3                                    7,079,424
│    │    └─MT5Block: 3-4                                    7,079,424
│    │    └─MT5Block: 3-5                                    7,079,424
│    │    └─MT5Block: 3-6                                    7,079,424
│    │    └─MT5Block: 3-7                                    7,079,424
│    │    └─MT5Block: 3-8                                    7,079,424
│    │    └─MT5B

In [21]:
article_text = r'''
연차사용이 자유롭고 분위기가 느슨하다. 유연근무제 도입했다.
연봉은 최악이며 미래먹거리가 없으며 저마다 처신하기 바쁨.
제발 정신좀차리고 주인의식 좀 느끼세요. 그대들 배부르다고 직원들도 배부른줄아나?
자유로운 연차 사용으로 원할때 사용하면 되며, 직원들을 위해서 다양한 소통 문화를 만들려고 함.
개인 연차나 비활동시에도 시스템에서 바로 확인되었으면 좋겠고, 연차 시 확인이 안되서 업무 전화가 많이 옴.
회사 직원들의 의견을 더 들어주셨으면 좋겠고, 적극적으로 반영해주셨으면 좋겠습니다.
월급 안밀리고 밥나오고 연차를 눈치없이 쓸 수 있음.
오래다니면 다닐수록 일이 더 쉬워짐.
직원을 홀대하고 연봉을 올려주지 않는 반면 경력직들은 직전연봉 다 챙겨줘 가며 입사시킴.
바라는 것도 없고 그냔 이대로만 해주세요. 잘하고 있다면서요?
연차 자율제도와 유연근무 제도로 가정에 더 충실할 수 있고 삶이 생긴다.
한량? 들이 많고 일을 하려는 의지들이 약하며 다들 불만만 많은 상황인데 인사정책은 더 역효과를 불러옴.
본인들 평가나 성과를 위한 업무를 하지말고 밑에 사람들을 돌보아야 될듯.
'''

In [22]:
article_text = article_text.replace('\n', ' ').strip()

In [23]:
get_lang_id = lambda lang: tokenizer._convert_token_to_id(
    model.config.task_specific_params["langid_map"][lang][1]
) 

target_lang = "korean" # for a list of available language names see below

input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512,
)["input_ids"]

Keyword arguments {'device': 'cuda'} not recognized.


In [29]:
model.to("cuda")
input_ids = input_ids.to("cuda")

In [30]:
output_ids = model.generate(
    input_ids=input_ids,
    decoder_start_token_id=get_lang_id(target_lang),
    max_length=84,
    no_repeat_ngram_size=1,
    num_beams=4,
)[0]

In [31]:
pred = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

In [32]:
print(pred)

<extra_id_70> 회사에서 연차 사용이 자유롭고 분위기가 느슨하다.


In [33]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [ ]:
def get_sae4k_df(path='/home/parking/ml/data/MiniProj/data/sae4k/sae4k_v2.txt'):
    

In [ ]:
class Sae4KDataset(Dataset):
    def __init__(self, tokenizer, df):
        self.__tokenizer = tokenizer
        self.__df = df